In [1]:
from joblib import Memory

memory = Memory(".cache", verbose=0)

In [2]:
import torch
import random

random_seed = 42
random.seed(random_seed)
 
# Set a random seed for PyTorch (for GPU as well)
torch.manual_seed(random_seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(random_seed)

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', DEVICE)

Using device: cuda


In [3]:
from transformers import BertTokenizer, BertModel

# @memory.cache
def get_tokenizer_model():
    bert_tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
    bert_model = BertModel.from_pretrained("bert-base-cased").to(DEVICE)

    return bert_tokenizer, bert_model

In [4]:
from pathlib import Path
import itertools
from natsort import natsorted

# @memory.cache
def read_problem_files(problem_folder, start=0, stop=None):
    # print(f"{start=} {stop=}")
    problems = []
    files = itertools.islice(natsorted(Path(problem_folder).glob('problem-*.txt')), start, stop)
    for problem_file in files:
        # print(f"{problem_file=}")
        # number = problem_file.name[len("problem-") : -len(".txt")]
        with open(problem_file, 'r', encoding="utf8") as fh:
            problems.append(fh.readlines())
    return problems

In [5]:
from evaluation.evaluator import read_ground_truth_files

# @memory.cache
def cached_read_ground_truth(x):
    return read_ground_truth_files(x)

# ground_truth = cached_read_ground_truth("pan21/train")

In [6]:
def get_embeddings(paragraph):
    bert_tokenizer, bert_model = get_tokenizer_model()
    # print(f"{paragraph=}")
    # Tokenize the sentence
    # tokens = bert_tokenizer.tokenize(paragraph)
    # token_ids = bert_tokenizer.convert_tokens_to_ids(tokens)

    # # Convert token IDs to tensor
    # token_tensor = torch.tensor([token_ids]).to(DEVICE)
    # segment_ids = [0] * len(token_ids)
    # segment_tensor = torch.tensor([segment_ids]).to(DEVICE)

    # # Get BERT model output
    # with torch.no_grad():
    #     outputs = bert_model(token_tensor, segment_tensor).last_hidden_state 

    # # Extract word embeddings from BERT model output
    # print(f"{outputs=}")
    # return outputs.cpu()
    inputs = bert_tokenizer(paragraph, return_tensors='pt', padding=True, truncation=True)

    # Move inputs to GPU
    inputs = {key: value.to(DEVICE) for key, value in inputs.items()}

    # Step 4: Generate embeddings
    with torch.no_grad():
        outputs = bert_model(**inputs)

    # The last hidden state contains the embeddings for each token
    embeddings = outputs.last_hidden_state

    # Step 5: Move embeddings to CPU
    # embeddings_cpu = embeddings.cpu()
    # assert embeddings.shape[0] != 0, f"{embeddings.shape=} {paragraph=}"
    # print(f"{embeddings.shape=} {paragraph=}")
    return embeddings

In [7]:
get_embeddings("Example of a paragraph")

tensor([[[ 0.2262,  0.0644,  0.0983,  ..., -0.1795,  0.4080,  0.2163],
         [-0.0450,  0.0464,  0.1558,  ...,  1.0553,  0.7027,  0.4606],
         [-0.2992, -0.1098, -0.1588,  ...,  0.4847,  0.0786,  0.5382],
         ...,
         [-0.0126,  0.2520, -0.2888,  ...,  0.4474,  0.5018,  0.6124],
         [ 0.3478,  0.1949, -0.2288,  ..., -0.0712,  0.3576,  0.3953],
         [ 0.2155,  0.4695, -0.6410,  ...,  0.6808,  1.1330, -0.1480]]],
       device='cuda:0')

In [8]:
import torch
import numpy as np

max_input_length = 256

def pad_paragraph(paragraph_embedding, desired_length):
    d1, d2, d3 = paragraph_embedding.shape
    # print(f"{paragraph_embedding.shape=}")

    target = torch.zeros(d1, desired_length, d3)
    # print(f"{target.shape=}")
    target[:, :d2, :] = paragraph_embedding

    return target


def get_paragraph_pairs(problem_text):
    # print(problem_text)
    paragraph_embeddings = [pad_paragraph(get_embeddings(para[:max_input_length]), max_input_length) for para in problem_text]
    # print(f"{[paras.shape for paras in paragraph_embeddings]=}")
    # print(f"{[x.shape for x in paragraph_embeddings]}")
    # print(f"{len(paragraph_embeddings)=}")
    pairs = itertools.combinations(paragraph_embeddings, 2)
    return [torch.flatten(torch.stack(pair, dim=2), start_dim=1, end_dim=2) for pair in pairs]


In [9]:
from tqdm.notebook import tqdm

# #@memory.cache
def get_problem_embeddings(problems, verbose=False):
    if verbose:
        return [get_paragraph_pairs(problem_text) for problem_text in tqdm(problems)]
    else:
        return [get_paragraph_pairs(problem_text) for problem_text in problems]

In [10]:
def get_simple_ground_truth(ground_truth, problem_numbers):
    simple_ground_truth = []
    for num in problem_numbers:
        task_3_ground_truth = ground_truth[f"problem-{num}"]["paragraph-authors"]
        simple_ground_truth.append(task_3_ground_truth)
    return simple_ground_truth


# simple_ground_truth = get_simple_ground_truth(ground_truth, range(1, len(problems_embed)))

In [11]:
def get_task_3_ground_truth(simple_ground_truth):
    # print(f"{len(simple_ground_truth)=}")
    task_gt = []
    for problem in simple_ground_truth:
        problem_gt = []
        for author1, author2 in itertools.combinations(problem, 2):
            problem_gt.append(int(author1 != author2))
        task_gt.append(problem_gt)
    # print(f"{len(task_gt)=}")
    return task_gt

# task_3_ground_truth = get_task_3_ground_truth(simple_ground_truth)

In [12]:
# TODO: Invert the function get_task_3_ground_truth. Our model will output a bunch of binary labels which need to be converted to the task 3 ground truth format
# Ground truth format (gtf): [1, 2, 2, 2, 2, 3, 2, 2]
# Binary labels for comparisons (bl): [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0]
# Each binary label is the result of comparing two paragraphs. 1 means there was an author change, 0 means there was no author change
# For example, bl[0], is the result of comparing gtf[0]=1 and gtf[1]=2. 1 != 2, therefore bl[0] = 1. bl[1]=1 is the result of gtf[0] == gtf[2] (1 == 2)
def get_simple_ground_truth_from_task_3(task_3_ground_truth):
    simple_gt = []
    for problem in task_3_ground_truth:
        # k = n*(n-1)/2
        # n**2 - n - 2k = 0
        coeff = [1, -1, len(problem) * -2]
        roots = np.roots(coeff)
        gt_length = int(roots[roots > 0][0])
        # print(gt_length)

        gt = np.zeros(gt_length, dtype=np.uint8)
        gt[0] = 1
        for i in range(1, gt_length):
            # loop for gt[i]
            num_comparisons = i
            pointer = i - 1
            modified_flag = False
            # print(f"{i=} {num_comparisons=} {pointer=}")
            for gt_i, j in enumerate(range(gt_length-2, 1, -1)[:num_comparisons]):
                # comparison between gt[gt_i] and gt[i]
                # print(f"{gt_i=} {j=} {pointer=} {task_3_ground_truth[pointer]=}")
                bin_label = task_3_ground_truth[pointer]
                if bin_label == 0:
                    # print(f"{gt[i]=} {gt[gt_i]=}")
                    gt[i] = gt[gt_i]
                    modified_flag = True
                    break

                pointer += j
            if not modified_flag:
                # print(f"No modified")
                gt[i] = np.max(gt) + 1
            # print(f"{gt}\n")
        simple_gt.append(gt)
    return simple_gt

# simple_ground_truth == get_simple_ground_truth_from_task_3(task_3_ground_truth)

In [13]:
def get_cuda_memory(i):
    if not torch.cuda.is_available():
        return 0

    total_memory = torch.cuda.get_device_properties(i).total_memory
    reserved_memory = torch.cuda.memory_reserved(i)
    allocated_memory = torch.cuda.memory_allocated(i)
    free_memory = total_memory - (reserved_memory + allocated_memory)
    
    return free_memory / (1024 ** 2) # MB

In [14]:
import math
from tensorflow.keras.utils import PyDataset
from pathlib import Path
from joblib import Parallel, delayed, parallel_config
import os

def flatten_problems(problems_list, squeeze=False):
        # [print(f"{pair=}") for problem in problems_list for pair in problem]
        return [pair.squeeze(0) if squeeze else pair for problem in problems_list for pair in problem]

class Pan21PyDataset(PyDataset):

    def __init__(self, x_path, y_set, file_path, batch_size=32,**kwargs):
        super().__init__(**kwargs)
        self.x_path, self.y = x_path, read_ground_truth_files(y_set)
        self.batch_size = batch_size

        self.file_path = Path(file_path)
        self.file_path.mkdir(parents=True, exist_ok=True)

    def __len__(self):
        # Return number of batches.
        return math.ceil(len(self.y) / self.batch_size)

    def get_data(x_path, y, low_idx, high_idx):
        # print(f"{self.x_path=}")
        embeddings = get_problem_embeddings(read_problem_files(x_path, low_idx, high_idx))
        gt = get_task_3_ground_truth(get_simple_ground_truth(y, range(low_idx+1, high_idx+1)))
        
        # print(f"{len(embeddings)=}")
        # print(f"{len(gt)=}")
        # print(f"{[len(embed) for embed in embeddings]=}")
        # print(f"{[len(g) for g in gt]=}")

        # assert np.all([len(e) == len(g) for e, g in zip(embeddings, gt)]), f"{[len(embed) for embed in embeddings]=}\n" + f"{[len(g) for g in gt]=}"

        batch_x = np.array(flatten_problems(embeddings, squeeze=True))
        batch_y = np.array(flatten_problems(gt))
        return batch_x, batch_y

    def to_file_helper(idx_path, i, x_path, y, batch_size):
        # print(f"{idx_path=}")
        low = i * batch_size
        # Cap upper bound at array length; the last batch may be smaller
        # if the total number of items is not a multiple of batch size.
        high = min(low + batch_size, len(y))
        # print(f"{low=} {high=}")
        batch_x, batch_y = Pan21PyDataset.get_data(x_path, y, low, high)
        # print(f"{batch_x.shape=}")
        np.savez_compressed(idx_path, batch_x=batch_x, batch_y=batch_y)

    def to_file(self, overwrite=False):
        free_memory = get_cuda_memory(0)
        n_jobs = min(free_memory // 850, os.cpu_count() - 2)
        # print(f"{n_jobs=} {free_memory=} {os.cpu_count()=}")
        args_for_jobs = []
        for i in range(len(self)):
            idx_path = self.file_path / f"{i}.npz"
            if not idx_path.exists() or overwrite:
                args_for_jobs.append((idx_path, i, self.x_path, self.y, self.batch_size))
        
        with parallel_config(backend='threading', n_jobs=n_jobs):
            Parallel()(delayed(Pan21PyDataset.to_file_helper)(*args) for args in tqdm(args_for_jobs))
        # for i in tqdm(range(len(self))):
        #     idx_path = self.file_path / f"{i}.npz"
        #     if not idx_path.exists() or (idx_path.exists() and overwrite):
        #         batch_x, batch_y = self.__getitem__(i, force_compute=True)
        #         np.savez_compressed(idx_path, batch_x=batch_x, batch_y=batch_y)

    def __getitem__(self, idx, force_compute=False):
        idx_path = self.file_path / f"{idx}.npz"
        # print(f"Pan21PyDataset {idx_path=}")
        if force_compute or not idx_path.exists():
            # print(f"{idx=}")
            # Return x, y for batch idx.
            low = idx * self.batch_size
            # Cap upper bound at array length; the last batch may be smaller
            # if the total number of items is not a multiple of batch size.
            high = min(low + self.batch_size, len(self.y))
            # print(f"{low=} {high=}")
            batch_x, batch_y = Pan21PyDataset.get_data(self.x_path, self.y, low, high)

            # print(f"{batch_x[0].shape=}")
            # print(f"{len(batch_x)=}")
            # print(f"{len(batch_y)=}")
        else:
            print(f"file exists")
            npzfile = np.load(idx_path)
            batch_x = npzfile['batch_x']
            batch_y = npzfile['batch_y']

        assert batch_x.shape[0] != 0, f"{idx=}: Dimension is 0 {batch_x.shape=} {batch_y.shape=}"
        assert batch_x.shape[0] == batch_y.shape[0], f"{idx=}: Dimension mismatch {batch_x.shape=} {batch_y.shape=}"

        return batch_x, batch_y

2024-06-03 19:56:47.180676: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-03 19:56:47.224840: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-03 19:56:47.993055: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [15]:
class Pan21FourierDataset(Pan21PyDataset):
    def __init__(self, x_set, y_set, file_path, batch_size, num_fourier_features, **kwargs):
        super().__init__(x_set, y_set, file_path, batch_size, **kwargs)
        self.num_fourier_features = num_fourier_features

    def __getitem__(self, idx, force_compute=False):
        idx_path = self.file_path / "fourier" / f"{idx}.npz"
        return Pan21FourierDataset.__getitem__helper(idx_path, idx, num_fourier_features=self.num_fourier_features, force_compute=force_compute)

    def __getitem__helper(idx_path, idx, num_fourier_features, force_compute=False):
        # idx_path = file_path / "fourier" / f"{idx}.npz"
        # print(f"Pan21FourierDataset {idx_path=}")

        # batch_x, batch_y = super().__getitem__(idx, force_compute)
        # print(f'{idx_path.parent / ".." / f"{idx}.npz"=}')
        embed_file = np.load(idx_path.parent / ".." / f"{idx}.npz")
        batch_x = embed_file['batch_x']
        batch_y = embed_file['batch_y']
        
        if num_fourier_features > 0:
            new_batch_x = batch_x.copy()
            
            num_features = len(batch_x[0])
            # print(f"{num_features=}")
            # 0:x will be BERT embeddings for paragraph 1
            # x:length/2 will be fourier features for paragraph 1
            num_non_fourier_features = (num_features - num_fourier_features) // 2
            para1_fourier_features_low, para1_fourier_features_high = num_non_fourier_features, num_features // 2
            para2_fourier_features_low, para2_fourier_features_high = num_features // 2 + num_non_fourier_features , num_features

            # print(f"{para1_fourier_features_low=} {para1_fourier_features_high=}")
            # print(f"{para2_fourier_features_low=} {para2_fourier_features_high=}")

            if force_compute or not idx_path.exists():
                para1_end = num_features//2
                for i, x in enumerate(batch_x):
                    para1_fft = np.real(np.fft.fft(x[:para1_end], axis=1))
                    para2_fft = np.real(np.fft.fft(x[para1_end:], axis=1))

                    new_batch_x[i, para1_fourier_features_low:para1_fourier_features_high] = para1_fft[:num_fourier_features//2]
                    new_batch_x[i, para2_fourier_features_low:para2_fourier_features_high] = para2_fft[:num_fourier_features//2]
            else:
                npzfile = np.load(idx_path)
                fourier_batch_x = npzfile['fourier_batch_x']

                _,j,_ = fourier_batch_x.shape

                new_batch_x[:, para1_fourier_features_low:para1_fourier_features_high, :] = fourier_batch_x[:, :num_fourier_features//2, :]
                new_batch_x[:, para2_fourier_features_low:para2_fourier_features_high, :] = fourier_batch_x[:, j//2:(j+num_fourier_features)//2, :]

            return new_batch_x, batch_y
        else:
            return batch_x, batch_y
    
    def save_file(idx_path, idx, num_fourier_features, force_compute=False):
        fourier_batch_x, _ = Pan21FourierDataset.__getitem__helper(idx_path, idx, num_fourier_features, force_compute)
        np.savez_compressed(idx_path, fourier_batch_x=fourier_batch_x)

    def to_file(self, overwrite=False):
        fourier_file_path = self.file_path / "fourier"
        fourier_file_path.mkdir(parents=True, exist_ok=True)

        n_jobs = os.cpu_count() // 2
        # print(f"{n_jobs=} {free_memory=} {os.cpu_count()=}")
        args_for_jobs = []
        for i in range(len(self)):
            idx_path = self.file_path / "fourier" / f"{i}.npz"
            if not idx_path.exists() or overwrite:
                args_for_jobs.append((idx_path, i, 512, True))
        
        with parallel_config(backend='threading', n_jobs=n_jobs):
            Parallel()(delayed(Pan21FourierDataset.save_file)(*args) for args in tqdm(args_for_jobs))

        

        # for i in tqdm(range(len(self))):
        #     idx_path = fourier_file_path / f"{i}.npz"
        #     if not idx_path.exists() or (idx_path.exists() and overwrite):
        #         fourier_batch_x, _ = self.__getitem__helper(i, num_fourier_features=512, force_compute=True)
        #         np.savez_compressed(idx_path, fourier_batch_x=fourier_batch_x)

In [16]:
class Pan21FourierFilterDataset(Pan21FourierDataset):
    def __init__(self, x_set, y_set, batch_size, num_fourier_features, cutoff_frequencies, **kwargs):
        super().__init__(x_set, y_set, batch_size, num_fourier_features, **kwargs)
        self.cutoff_frequencies = cutoff_frequencies
    
    def __getitem__(self, idx, force_compute=False):
        return self.__getitem__helper(idx, num_fourier_features=self.num_fourier_features, force_compute=force_compute)

In [17]:
batch_size = 32

Pan21PyDataset("pan21/train", "pan21/train", "train_ds", batch_size).to_file()
Pan21PyDataset("pan21/validation", "pan21/validation", "val_ds", batch_size).to_file()

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [18]:
# Make sure the files are good
def test_load(path, name):
    np.load(path)[name]

# _ = Parallel(n_jobs=-1)(delayed(test_load)(Path("train_ds") / f"{i}.npz", "batch_x") for i in tqdm(range(350)))

In [19]:
Pan21FourierDataset("pan21/train", "pan21/train", "train_ds", batch_size, num_fourier_features=512).to_file()
Pan21FourierDataset("pan21/validation", "pan21/validation", "val_ds", batch_size, num_fourier_features=512).to_file()

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [20]:
# _ = Parallel(n_jobs=-1)(delayed(test_load)(Path("train_ds") / "fourier" / f"{i}.npz", "fourier_batch_x") for i in tqdm(range(350)))

In [21]:
# import time

# start_idx = 0
# limit = 5
# # limit = len(train_ds)

# before = time.time()
# for i in range(start_idx, start_idx+limit):
#     batch_x, batch_y = train_ds.__getitem__(i, force_compute=True)
# after = time.time()
# for i in range(start_idx, start_idx+limit):
#     batch_x, batch_y = train_ds.__getitem__(i, force_compute=False)
# after_after = time.time()

# print(f"Compute: {round((after - before)/limit, 2)}s vs File read: {round((after_after - after)/limit, 2)}s")
# # Compute: 8.95s per batch
# # Compute with compression: ~17s
# # Uncompressed read: .7s per batch
# # Compressed read: 1.6s per batch
# # Compressed is 1/10 the size of uncompressed, but takes ~twice as long to precompute and save
# # Compressed 512D Fourier takes ~30s per batch
# # Compressed 512D Fourier is about 500-700MB per batch

In [22]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, TimeDistributed, Input, Flatten
from tensorflow.keras import optimizers, losses, metrics

# Code implementation of the RNN for sequence labeling
def create_rnn_model(num_labels, embedding_dim):
    model = Sequential()
    model.add(Input(shape=(max_input_length*2, embedding_dim)))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Flatten())
    model.add(Dense(num_labels, activation='sigmoid'))

    model.compile(
        optimizer=optimizers.RMSprop(),  # Optimizer
        # Loss function to minimize
        loss=losses.BinaryCrossentropy(),
        # List of metrics to monitor
        metrics=[metrics.BinaryAccuracy(), metrics.AUC()],
        jit_compile=True
    )

    return model
 
num_labels = 1
embedding_dim = 768

In [26]:
# Does using the frequency domain spectra provide usefule information?
import datetime
from keras.callbacks import CSVLogger

def train_model_num_ff(num_ff):
    model_name = model_dir / f"{num_ff}_{datetime.datetime.now().strftime('%Y_%m_%d-%I_%M_%S_%p')}"
    model = create_rnn_model(num_labels, embedding_dim)
    csv_logger = CSVLogger(f'{model_name}.log', separator=',', append=False)

    fourier_train_ds = Pan21FourierDataset("pan21/train", "pan21/train", "train_ds", batch_size, num_fourier_features=num_ff)
    fourier_val_ds = Pan21FourierDataset("pan21/validation", "pan21/validation", "val_ds", batch_size, num_fourier_features=num_ff)

    history = model.fit(
        fourier_train_ds,
        epochs=5,
        validation_data=fourier_val_ds,
        verbose=1,
        callbacks=[csv_logger]
    )

    model.save(f"{model_name}.keras")
# num_fourier_features = [0, 512//4, 512//2, 512]
num_fourier_features = [512//4, 512//2, 512]

model_dir = Path(f"models/num_fourier_features")
if not model_dir.exists():
    model_dir.mkdir(parents=True)

_ = Parallel(n_jobs=1)(delayed(train_model_num_ff)(num_ff) for num_ff in tqdm(num_fourier_features))

  0%|          | 0/3 [00:00<?, ?it/s]

Epoch 1/5


2024-06-03 20:05:05.974791: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:8: Filling up shuffle buffer (this may take a while): 1 of 8
2024-06-03 20:05:18.219029: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:8: Filling up shuffle buffer (this may take a while): 2 of 8
2024-06-03 20:05:30.573105: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:8: Filling up shuffle buffer (this may take a while): 3 of 8
2024-06-03 20:05:46.237509: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:8: Filling up shuffle buffer (this may take a while): 4 of 8
2024-06-03 20:05:57.518064: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:8: Filling up shuffle buffer (this may take a while): 5 of 8
2024-06-03 20:06:13.861988: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:8: Filling up shuffle buffer (this may take a while): 7 of 8
2024-06-03 20:06

  1/350 ━━━━━━━━━━━━━━━━━━━━ 9:20:47 96s/step - auc_1: 0.5141 - binary_accuracy: 0.4389 - loss: 0.9322

I0000 00:00:1717445196.578169  551937 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_12', 412 bytes spill stores, 416 bytes spill loads



...

350/350 ━━━━━━━━━━━━━━━━━━━━ 4665s 13s/step - auc_1: 0.6685 - binary_accuracy: 0.6238 - loss: 0.6446 - val_auc_1: 0.7055 - val_binary_accuracy: 0.6369 - val_loss: 0.6255
Epoch 3/5


2024-06-03 22:40:58.913941: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:8: Filling up shuffle buffer (this may take a while): 1 of 8
2024-06-03 22:41:11.468317: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:8: Filling up shuffle buffer (this may take a while): 2 of 8
2024-06-03 22:41:21.916732: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:8: Filling up shuffle buffer (this may take a while): 3 of 8
2024-06-03 22:41:39.841817: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:8: Filling up shuffle buffer (this may take a while): 5 of 8
2024-06-03 22:41:50.170768: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:8: Filling up shuffle buffer (this may take a while): 6 of 8


  1/350 ━━━━━━━━━━━━━━━━━━━━ 7:37:50 79s/step - auc_1: 0.7303 - binary_accuracy: 0.6389 - loss: 0.6521

2024-06-03 22:42:06.103454: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


233/350 ━━━━━━━━━━━━━━━━━━━━ 21:20 11s/step - auc_1: 0.7092 - binary_accuracy: 0.6509 - loss: 0.6168

In [68]:
# history = model.fit(
#     x_train,
#     y_train,
#     batch_size=50,
#     epochs=3,
#     # We pass some validation for
#     # monitoring validation loss and metrics
#     # at the end of each epoch
#     validation_data=(x_val, y_val),
# )
history = model.fit(
    train_ds,
    epochs=1,
    # We pass some validation for
    # monitoring validation loss and metrics
    # at the end of each epoch
    validation_data=val_ds,
    verbose=1,
)

import datetime

model.save(f"models/baseline_{datetime.datetime.now().strftime('%Y_%m_%d-%I_%M_%S_%p')}.keras")

350/350 ━━━━━━━━━━━━━━━━━━━━ 1320s 4s/step - auc: 0.8467 - binary_accuracy: 0.7743 - loss: 0.4627 - val_auc: 0.5286 - val_binary_accuracy: 0.5231 - val_loss: 1.1656


In [ ]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(x_test, y_test, batch_size=128)
print("test loss, test acc:", results)

# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print("Generate predictions for 3 samples")
predictions = model.predict(x_test[:3])
print("predictions shape:", predictions.shape)
